In [205]:
import os
import pandas as pd
pd.options.display.max_rows = 4000
import matplotlib.pyplot as plt
from typing import List, Tuple
import numpy as np
from scipy.signal import find_peaks
from sklearn.preprocessing import StandardScaler

import sys
sys.path.append("plotting_dir")
import plotting_feature_extraction
import importlib
importlib.reload(plotting_feature_extraction)

<module 'plotting_feature_extraction' from '/home/liam-bouffard/Desktop/LASSIE/plotting_dir/plotting_feature_extraction.py'>

In [206]:
curves_data = []
filename_list = []
for filename in os.listdir("data/cleaned_data"):
    df = pd.read_csv(f"data/cleaned_data/{filename}").drop(columns=["Unnamed: 0"])
    filename_list.append(filename)
    curves_data.append(df)
print(filename_list)
print(curves_data[0])

['WS23_L1_T1_P_10.csv', 'WS23_L3_T1_P_10.csv', 'WS23_L3_T1_P_6.csv', 'WS23_L3_T1_P_1.csv', 'WS23_L3_T1_P_8.csv', 'WS23_L1_T1_P_17.csv', 'WS23_L3_T1_P_31.csv', 'WS23_L1_T1_P_6.csv', 'WS23_L1_T1_P_7.csv', 'WS25_Aug5_Loc1A_T1_F17_1029.csv', 'WS25_Aug5_Loc1A_T1_F6_0930.csv', 'WS25_Aug6_Loc1A_T1_F32_0714.csv', 'WS25_Aug5_Loc1A_T1_F12_1010.csv', 'WS23_L3_T1_P_0.csv', 'WS23_L1_T1_P_2.csv', 'WS23_L1_T1_P_8.csv', 'WS25_Aug5_Loc1A_T1_F8_0955.csv', 'WS23_L3_T1_P_24.csv', 'WS25_Aug5_Loc1A_T1_F5_0929.csv', 'WS23_L3_T1_P_18.csv', 'WS25_Aug6_Loc1A_T1_F20_0643_attempt3.csv', 'WS23_L1_T1_P_19.csv', 'WS23_L2_T1_P_2.csv', 'WS23_L1_T1_P_16.csv', 'WS23_L1_T1_P_1.csv', 'WS25_Aug5_Loc1A_T1_F24_1057.csv', 'WS23_L3_T1_P_4.csv', 'WS25_Aug5_Loc1A_T1_F16_1028.csv', 'WS23_L1_T1_P_18.csv', 'WS23_L3_T1_P_13.csv', 'WS23_L3_T1_P_16.csv', 'WS25_Aug6_Loc1A_T1_F23_0656.csv', 'WS23_L2_T1_P_13.csv', 'WS23_L2_T1_P_5.csv', 'WS23_L1_T1_P_3.csv', 'WS25_Aug6_Loc1A_T1_F18_0624.csv', 'WS23_L1_T1_P_22.csv', 'WS23_L2_T1_P_7.csv', '

# Find Curve Shape

In [207]:
def find_curve_shape(df):
    x = df['depth'].to_numpy()
    y = df['resistance'].to_numpy()
    # create evenly spaces points
    chord = np.linspace(df['resistance'].iloc[0], df['resistance'].iloc[-1], num=len(df['depth']))
    y_diff = y - chord # makes chord the x-axis, any y_points above chord are pos, below are neg
    return np.trapezoid(y=y_diff, x=x)

# Find Force-Drop

In [208]:
def find_force_drop_subranges(df: pd.DataFrame, percent_of_max_resistance: float):
    down_moves_subrange_list = []
    resistance_max = df['resistance'].max()
    min_drop_size = resistance_max * percent_of_max_resistance
    curr_peak_idx = 0
    curr_trough_idx = 0
    in_drop_subrange = 0 # bool

    for idx in range(1, len(df['resistance'])):
        curr_peak = df['resistance'].iloc[curr_peak_idx]
        curr_trough = df['resistance'].iloc[curr_trough_idx]
        curr_resistance = df['resistance'].iloc[idx]

        if curr_resistance >= curr_peak and in_drop_subrange:
            in_drop_subrange = 0
            down_moves_subrange_list.append((curr_peak_idx, curr_trough_idx))

        if curr_resistance >= curr_peak:
            curr_peak_idx = idx
            curr_trough_idx = idx
        elif curr_peak - curr_resistance >= min_drop_size and curr_resistance < curr_trough: 
            in_drop_subrange = 1
            curr_trough_idx = idx
    
    if in_drop_subrange: down_moves_subrange_list.append((curr_peak_idx, curr_trough_idx)) 
            
    return down_moves_subrange_list

def find_largest_force_drop(df: pd.DataFrame, subrange_list: List[Tuple]):
    curr_max_drop_size = 0
    curr_max_subrange_idxs = (0,0)
    for subrange_start, subrange_end in subrange_list:
        subrange_diff = df['resistance'].iloc[subrange_start] - df['resistance'].iloc[subrange_end]
        if subrange_diff > curr_max_drop_size: 
            curr_max_drop_size = subrange_diff
            curr_max_subrange_idxs = (subrange_start, subrange_end)
    return curr_max_drop_size, curr_max_subrange_idxs

def plot(curves_data: List[pd.DataFrame], plot_idx_range: List[int], title: str = 'Depth vs Resistance'):

    all_depth_resistance_data = pd.concat(curves_data, axis=0, ignore_index=True)
    gloabl_max_depth = all_depth_resistance_data['depth'].max()
    gloabl_max_resistance = all_depth_resistance_data['resistance'].max()

    for idx in plot_idx_range:

        print(f"plot idx: {idx}")

        df = curves_data[idx]
        percent = 0.1
        subranges = find_force_drop_subranges(df, percent)
        print(f"max_resistance: {df['resistance'].max()}")
        print(f"subranges: {[(float(df['resistance'].iloc[start]), float(df['resistance'].iloc[end])) for start, end in subranges]}")
   
        plt.figure(figsize=(5, 3))
        plt.xlim(0,gloabl_max_depth)
        plt.ylim(0,gloabl_max_resistance)
        plt.plot([0,df['depth'].max()], [0,df['resistance'].iloc[df['depth'].values.argmax()]], color='red')
        plt.plot([0,df['depth'].max()], [0,df['resistance'].iloc[df['depth'].values.argmax()]], color='red')
        plt.plot([0,df['depth'].max()], [0,df['resistance'].iloc[df['depth'].values.argmax()]], color='red')

        # Plot full depth vs resistance line
        plt.plot(df['depth'], df['resistance'],linestyle='-')
        plt.xlabel('Depth (m)')
        plt.ylabel('Resistance (N)')
        plt.title(f"{title} - Plot {idx}")
        # plt.grid(True)
        plt.tight_layout()
        plt.show()

In [209]:
def find_koverall(curve):
    return np.polyfit(curve['depth'], curve['resistance'], 1)[0]

def find_k2cm(curve):
    curve = curve[curve['depth'] <= 0.02]
    if curve['depth'].max() < 0.019: return 0 # 0.019 bc most data doesn't have a perfect 0.02 depth
    return np.polyfit(curve['depth'], curve['resistance'], 1)[0]

def find_ksurface(curve):
    pass

from sklearn.metrics import r2_score
def find_rsquared(curve):
    coeffs = np.polyfit(curve['depth'], curve['resistance'], 1)
    predicted = np.polyval(coeffs, curve['depth'])
    actual = curve['resistance']
    return r2_score(actual, predicted)

def find_heterogenity(curve):
    pass

def find_work2cm(curve):
    if curve['resistance'].max() < 0.2: return 0
    depth_m = curve['depth'] * 1000.0  # convert m → mm
    work = np.trapezoid(curve['resistance'], depth_m)
    return work 

def find_Fpeak(curve):
    return curve['resistance'].max()

def find_dpeak(curve):
    return curve.loc[curve['resistance'].idxmax(), 'depth']

def find_Fbasin(curve):
    pass

def find_dbasin(curve):
    pass

# Choose Features
IMPORTANT: If you add variables here, you must add the names to the top of plotting.py file as well

In [210]:
def extract_simple_features(df):
    res = df["resistance"]
    dep = df["depth"]
    max_res_drop_val, max_res_drop_subrange_idxs = find_largest_force_drop(df, find_force_drop_subranges(df, 0.001))

    return pd.DataFrame({
        # "curve_overall_slope": [res.max() / dep.max()],
        "depth_max": [dep.max()],
        # "force_max": [res.max()],
        # "num_peaks": len(find_force_drop_subranges(df,0.1)) / dep.max(),
        "largest_force_drop_size": max_res_drop_val,
        # "largest_force_drop_dep": dep.loc[max_res_drop_subrange_idxs[0]],
        # "largest_force_drop_res": res.loc[max_res_drop_subrange_idxs[0]],
        # "curve_shape": find_curve_shape(df),
# 
        "curve_first_quarter_slope": (res.loc[round(0.25 * len(res))] - res.min()) / (dep.loc[round(0.25 * len(dep))] - dep.min()),
        # "curve_second_quarter_slope": (res.loc[round(0.50 * len(res))] - res.loc[round(0.25 * len(res))]) / (dep.loc[round(0.50 * len(dep))] - dep.loc[round(0.25 * len(dep))]),
        # "curve_third_quarter_slope": (res.loc[round(0.75 * len(res))] - res.loc[round(0.50 * len(res))]) / (dep.loc[round(0.75 * len(dep))] - dep.loc[round(0.50 * len(dep))]),
        # "curve_fourth_quarter_slope": (res.max() - res.loc[round(0.75 * len(res))]) / (dep.max() - dep.loc[round(0.75 * len(dep))]),
# 
        # resistance statistics
        # "force_quartile_1": res.quantile(0.25),
        # "force_quartile_2": res.quantile(0.50),
        # "force_quartile_3": res.quantile(0.75),
        # "force_variance": res.var(),
        "force_mean": res.mean(),
        # "force_skew": res.skew(), # measurs asymmetry of a distribution around it's mean
        # "force_kurtosis": res.kurt(), # descirbes tailedness or peakedness of a distribution
# 
        # USC's Feature Set
        # "k_overall": find_koverall(df),
        # "curve_2cm_slope": find_k2cm(df),
        "rsquared": find_rsquared(df),
    })


# shape (n,m) where n is number of df and m is extracted feaetures
representation_list = []
for i, df in enumerate(curves_data):
    extracted_simple_features = extract_simple_features(df)
    representation_list.append(extracted_simple_features)
print(f"{representation_list[0].columns}")

Index(['depth_max', 'largest_force_drop_size', 'curve_first_quarter_slope',
       'force_mean', 'rsquared'],
      dtype='object')


# Examples of what Features Are

In [211]:
# # here use plot feature extraction
# feature_names = ['largest_force_drop_size', 'force_mean', 'force_quartile_2',
#        'curve_first_quarter_slope', 'force_quartile_3', 'rsquared',
#        'force_quartile_1', 'force_variance', 'force_max', 'curve_2cm_slope',
#        'depth_max', 'curve_overall_slope', 'largest_force_drop_force',
#        'num_peaks', 'curve_third_quarter_slope', 'curve_shape',
#        'largest_force_drop_dep', 'curve_second_quarter_slope', 'force_skew',
#        'curve_fourth_quarter_slope', 'force_kurtosis']
# # plotting_feature_extraction.plot_feature_selection_seperately(
# #     feature_names, curves_data, 0,
# # )
# plotting_feature_extraction.plot_feature_selection(
#     feature_names, curves_data, 4,
# )

# Save the names of the numerical feature's used for clustering.
This is imported in plotting.py

In [212]:
numerical_feature_names = pd.Series(representation_list[0].columns.tolist(), name='numerical_feature_names')
numerical_feature_names.to_csv('data/numerical_feature_names.csv', index=False)

# Visualize Extracted Features Histograms

In [213]:
representation_df = pd.concat(representation_list, axis=0)

# def plot_feature_dist(representation_df):
#     # for col in representation_df.columns:
#     # x, y = plotting_feature_extraction.find_oriented_subplot_dims(len(representation_df.columns))
#     x, y = plotting_feature_extraction.find_subplot_dims_orientation(len(representation_df.columns))
#     fig, axs = plt.subplots(x,y,figsize=(round(x*2), round(y*5)))
#     # fig.suptitle('Feature Histograms')
#     for i, ax in enumerate(axs.flatten()):
#         if i > len(representation_df.columns)-1: break
#         col = representation_df.columns[i]
#         ax.hist(representation_df[col], bins=60, density=True)
#         ax.set_title(f"{col.title()} Histogram", fontsize=12)
#     plt.tight_layout()
#     plt.show()

# plot_feature_dist(representation_df)

# Scale Features

In [214]:
def transform_features(df):
    df = df.copy()  # avoid modifying original
    scaler = StandardScaler()
    df[df.columns] = scaler.fit_transform(df[df.columns])
    return df

scaled_representations = transform_features(representation_df)
# plot_feature_dist(scaled_representations)


# Add Meta-Data

In [215]:
scaled_representations['filenames'] = filename_list

### Add Marion's y_labels

In [216]:
marions_filenames_to_ylabels = {
    "WS23_L2_T1_P_0.csv": 'LS', "WS23_L2_T1_P_1.csv": 'LS', "WS23_L2_T1_P_2.csv": 'LS', "WS23_L2_T1_P_3.csv": 'ES-D', "WS23_L2_T1_P_4.csv": "LS", "WS23_L2_T1_P_5.csv": "ES-D",
    "WS23_L2_T1_P_6.csv": "ES-D", "WS23_L2_T1_P_7.csv": "ES-D", "WS23_L2_T1_P_8.csv": "ES-DB", "WS23_L2_T1_P_9.csv": "ES-S", "WS23_L2_T1_P_10.csv": "ES-D", "WS23_L2_T1_P_11.csv": 'ES-D',
    "WS23_L2_T1_P_12.csv": 'ES-S', "WS23_L2_T1_P_13.csv": 'ES-D', "WS23_L2_T1_P_14.csv": 'ES-S', "WS23_L2_T1_P_15.csv": 'ES-D', "WS23_L2_T1_P_16.csv": 'ES-DB', "WS23_L2_T1_P_17.csv": 'ES-S',

    "WS23_L3_T1_P_0.csv": 'LS', "WS23_L3_T1_P_1.csv": 'ES-B', "WS23_L3_T1_P_2.csv": 'ES-B', "WS23_L3_T1_P_3.csv": 'ES-S',
    "WS23_L3_T1_P_4.csv": 'ES-B', "WS23_L3_T1_P_5.csv": 'ES-B', "WS23_L3_T1_P_6.csv": 'ES-BW', "WS23_L3_T1_P_7.csv": 'ES-B',
    "WS23_L3_T1_P_8.csv": 'ES-BW', "WS23_L3_T1_P_9.csv": 'F', "WS23_L3_T1_P_10.csv": 'ES-D', "WS23_L3_T1_P_11.csv": 'ES',
    "WS23_L3_T1_P_12.csv": 'F', "WS23_L3_T1_P_13.csv": 'F', "WS23_L3_T1_P_14.csv": 'ES-D', "WS23_L3_T1_P_15.csv": 'ES-D',
    "WS23_L3_T1_P_16.csv": 'ES-B', "WS23_L3_T1_P_17.csv": 'ES', "WS23_L3_T1_P_18.csv": 'ES-S-Plates', "WS23_L3_T1_P_19.csv": 'ES-S-Plates',
    "WS23_L3_T1_P_20.csv": 'LS/F', "WS23_L3_T1_P_21.csv": 'ES', "WS23_L3_T1_P_22.csv": 'ES', "WS23_L3_T1_P_23.csv": 'LS'
}
print(scaled_representations.columns)
scaled_representations['marions_ylabels'] = scaled_representations['filenames'].map(marions_filenames_to_ylabels)


Index(['depth_max', 'largest_force_drop_size', 'curve_first_quarter_slope',
       'force_mean', 'rsquared', 'filenames'],
      dtype='object')


### Add label creation meta-data

In [217]:


# popcorn_filenames = [
#     # 'WS23_L2_T1_P_3.csv', 'WS23_L2_T1_P_5.csv', 'WS23_L2_T1_P_6.csv', 'WS23_L2_T1_P_7.csv',
#     # 'WS23_L2_T1_P_8.csv', 'WS23_L2_T1_P_10.csv', 'WS23_L2_T1_P_11.csv', 'WS23_L2_T1_P_12.csv',
#     # 'WS23_L2_T1_P_13.csv', 'WS23_L2_T1_P_14.csv', 'WS23_L2_T1_P_15.csv', 'WS23_L2_T1_P_16.csv', 
#     # 'WS23_L2_T1_P_17.csv', 'WS23_L2_T1_P_18.csv', 'WS23_L2_T1_P_19.csv', 'WS23_L2_T1_P_20.csv', 
#     # 'WS23_L2_T1_P_21.csv', 'WS23_L2_T1_P_22.csv', 'WS23_L2_T1_P_23.csv', 'WS23_L2_T1_P_24.csv', 
#     # 'WS23_L2_T1_P_25.csv',
#     # 
#     'WS23_L3_T1_P_0.csv', 'WS23_L3_T1_P_1.csv', 'WS23_L3_T1_P_2.csv', 'WS23_L3_T1_P_3.csv', 
#     'WS23_L3_T1_P_4.csv', 'WS23_L3_T1_P_5.csv', 'WS23_L3_T1_P_6.csv', 'WS23_L3_T1_P_7.csv', 
#     'WS23_L3_T1_P_8.csv', 'WS23_L3_T1_P_9.csv', 'WS23_L3_T1_P_10.csv', 'WS23_L3_T1_P_11.csv', 
#     'WS23_L3_T1_P_12.csv', 'WS23_L3_T1_P_13.csv', 'WS23_L3_T1_P_14.csv', 'WS23_L3_T1_P_15.csv', 
#     'WS23_L3_T1_P_16.csv', 'WS23_L3_T1_P_17.csv', 'WS23_L3_T1_P_18.csv', 'WS23_L3_T1_P_19.csv', 
#     'WS23_L3_T1_P_20.csv', 'WS23_L3_T1_P_21.csv', 'WS23_L3_T1_P_22.csv', 'WS23_L3_T1_P_23.csv']

# popcorn = [
#     # False,False,False,False,
#     # True,False,False,False,
#     # False,

#     False,True,True,False,
#     True,True,True,True,
#     True,False,False,False,
#     False,False,False,False,
#     False,False,False,False,
#     False,False,False,False]

# filenames_to_popcorn = dict(zip(popcorn_filenames, popcorn))
# scaled_representations['popcorn'] = scaled_representations['filenames'].map(filenames_to_popcorn)


### Add feet from dune

In [218]:
filenames_list = ['WS23_L1_T1_P_0.csv', 'WS23_L1_T1_P_1.csv', 'WS23_L1_T1_P_2.csv', 'WS23_L1_T1_P_3.csv', 'WS23_L1_T1_P_4.csv', 'WS23_L1_T1_P_5.csv', 'WS23_L1_T1_P_6.csv', 'WS23_L1_T1_P_7.csv', 'WS23_L1_T1_P_8.csv', 'WS23_L1_T1_P_9.csv', 'WS23_L1_T1_P_10.csv', 'WS23_L1_T1_P_11.csv', 'WS23_L1_T1_P_12.csv', 'WS23_L1_T1_P_13.csv', 'WS23_L1_T1_P_14.csv', 'WS23_L1_T1_P_15.csv', 'WS23_L1_T1_P_16.csv', 'WS23_L1_T1_P_17.csv', 'WS23_L1_T1_P_18.csv', 'WS23_L1_T1_P_19.csv', 'WS23_L2_T1_P_0.csv', 'WS23_L2_T1_P_1.csv', 'WS23_L2_T1_P_2.csv', 'WS23_L2_T1_P_3.csv', 'WS23_L2_T1_P_4.csv', 'WS23_L2_T1_P_5.csv', 'WS23_L2_T1_P_6.csv', 'WS23_L2_T1_P_7.csv', 'WS23_L2_T1_P_8.csv', 'WS23_L2_T1_P_9.csv', 'WS23_L2_T1_P_10.csv', 'WS23_L2_T1_P_11.csv', 'WS23_L2_T1_P_12.csv', 'WS23_L2_T1_P_13.csv', 'WS23_L2_T1_P_14.csv', 'WS23_L2_T1_P_15.csv', 'WS23_L2_T1_P_16.csv', 'WS23_L2_T1_P_17.csv', 'WS23_L2_T2_P_0.csv', 'WS23_L2_T2_P_1.csv', 'WS23_L2_T2_P_2.csv', 'WS23_L2_T2_P_3.csv', 'WS23_L2_T2_P_4.csv', 'WS23_L3_T1_P_0.csv', 'WS23_L3_T1_P_1.csv', 'WS23_L3_T1_P_2.csv', 'WS23_L3_T1_P_3.csv', 'WS23_L3_T1_P_4.csv', 'WS23_L3_T1_P_5.csv', 'WS23_L3_T1_P_6.csv', 'WS23_L3_T1_P_7.csv', 'WS23_L3_T1_P_8.csv', 'WS23_L3_T1_P_9.csv', 'WS23_L3_T1_P_10.csv', 'WS23_L3_T1_P_11.csv', 'WS23_L3_T1_P_12.csv', 'WS23_L3_T1_P_13.csv', 'WS23_L3_T1_P_14.csv', 'WS23_L3_T1_P_15.csv', 'WS23_L3_T1_P_16.csv', 'WS23_L3_T1_P_17.csv', 'WS23_L3_T1_P_18.csv', 'WS23_L3_T1_P_19.csv', 'WS23_L3_T1_P_20.csv', 'WS23_L3_T1_P_21.csv', 'WS23_L3_T1_P_22.csv', 'WS23_L3_T1_P_23.csv']
distances_list = [0, 3, 5.5, 10.5, 12, 14, 38, 41.5, 44, 47, 51, 90, 93, 96, 102, 107, 151, 152, 153, 170, 3.25, 0, 6, 13, 16, 19, 24, 64, 67, 72, 75, 87, 88, 88, 91, 95, 112, 116, 0, 4, 11, 20, 27, 0, 10, 16, 19, 40, 49, 98, 160, 161, 187, 188, 229, 253, 254, 255, 308, 317, 318, 353, 357, 363, 369, 384, 389]

filenames_to_distances = dict(zip(filenames_list, distances_list))
scaled_representations['distances'] = scaled_representations['filenames'].map(filenames_to_distances)

# Meta Data

In [219]:

data_order = ['filenames', 'popcorn', 'clump']
# TODO create into a df then concat with map to main df
meta_data = np.array([
    ['WS23_L1_T1_P_0.csv', None, None],
    ['WS23_L1_T1_P_1.csv', None, None],
    ['WS23_L1_T1_P_2.csv', None, None],
    ['WS23_L1_T1_P_3.csv', None, None],
    ['WS23_L1_T1_P_4.csv', None, None],
    ['WS23_L1_T1_P_5.csv', None, None],
    ['WS23_L1_T1_P_6.csv', None, None],
    ['WS23_L1_T1_P_7.csv', None, None],
    ['WS23_L1_T1_P_8.csv', None, None],
    ['WS23_L1_T1_P_9.csv', None, None],
    ['WS23_L1_T1_P_10.csv', None, None],
    ['WS23_L1_T1_P_11.csv', None, None],
    ['WS23_L1_T1_P_12.csv', None, None],
    ['WS23_L1_T1_P_13.csv', None, None],
    ['WS23_L1_T1_P_14.csv', None, None],
    ['WS23_L1_T1_P_15.csv', None, None],
    ['WS23_L1_T1_P_16.csv', None, None],
    ['WS23_L1_T1_P_17.csv', None, None],
    ['WS23_L1_T1_P_18.csv', None, None],
    ['WS23_L1_T1_P_19.csv', None, None],
    ['WS23_L2_T1_P_0.csv', None, None],
    ['WS23_L2_T1_P_1.csv', None, None],
    ['WS23_L2_T1_P_2.csv', None, None],
    ['WS23_L2_T1_P_3.csv', False, None],
    ['WS23_L2_T1_P_4.csv', None, None],
    ['WS23_L2_T1_P_5.csv', False, None],
    ['WS23_L2_T1_P_6.csv', False, None],
    ['WS23_L2_T1_P_7.csv', False, None],
    ['WS23_L2_T1_P_8.csv', True, None],
    ['WS23_L2_T1_P_9.csv', None, None],
    ['WS23_L2_T1_P_10.csv', False, None],
    ['WS23_L2_T1_P_11.csv', False, None],
    ['WS23_L2_T1_P_12.csv', False, None],
    ['WS23_L2_T1_P_13.csv', False, None],
    ['WS23_L2_T1_P_14.csv', False, None],
    ['WS23_L2_T1_P_15.csv', False, None],
    ['WS23_L2_T1_P_16.csv', True, None],
    ['WS23_L2_T1_P_17.csv', False, None],
    ['WS23_L2_T1_P_18.csv', True, None],
    ['WS23_L2_T1_P_19.csv', True, None],
    ['WS23_L2_T1_P_20.csv', True, None],
    ['WS23_L2_T1_P_21.csv', True, None],
    ['WS23_L2_T1_P_22.csv', True, None],
    ['WS23_L2_T1_P_23.csv', False, None],
    ['WS23_L2_T1_P_24.csv', False, None],
    ['WS23_L2_T1_P_25.csv', False, None],
    ['WS23_L2_T2_P_0.csv', None, None],
    ['WS23_L2_T2_P_1.csv', None, None],
    ['WS23_L2_T2_P_2.csv', None, None],
    ['WS23_L2_T2_P_3.csv', None, None],
    ['WS23_L2_T2_P_4.csv', None, None],
    ['WS23_L3_T1_P_0.csv', None, False],
    ['WS23_L3_T1_P_1.csv', True, False],
    ['WS23_L3_T1_P_2.csv', True, False],
    ['WS23_L3_T1_P_3.csv', None, False],
    ['WS23_L3_T1_P_4.csv', True, False],
    ['WS23_L3_T1_P_5.csv', True, False],
    ['WS23_L3_T1_P_6.csv', True, False],
    ['WS23_L3_T1_P_7.csv', True, False],
    ['WS23_L3_T1_P_8.csv', True, False],
    ['WS23_L3_T1_P_9.csv', False, False],
    ['WS23_L3_T1_P_10.csv', False, True],
    ['WS23_L3_T1_P_11.csv', False, True],
    ['WS23_L3_T1_P_12.csv', False, False],
    ['WS23_L3_T1_P_13.csv', False, False],
    ['WS23_L3_T1_P_14.csv', False, True],
    ['WS23_L3_T1_P_15.csv', False, True],
    ['WS23_L3_T1_P_16.csv', False, True],
    ['WS23_L3_T1_P_17.csv', False, True],
    ['WS23_L3_T1_P_18.csv', False, False],
    ['WS23_L3_T1_P_19.csv', False, False],
    ['WS23_L3_T1_P_20.csv', False, False],
    ['WS23_L3_T1_P_21.csv', False, False],
    ['WS23_L3_T1_P_22.csv', False, False],
    ['WS23_L3_T1_P_23.csv', None, False]
])

meta_data_df = pd.DataFrame(meta_data, columns = data_order)
scaled_representations = scaled_representations.merge(meta_data_df, on='filenames', how='left')

# Save representation

In [220]:
scaled_representations.to_csv(f"data/features.csv", index=False)
print(scaled_representations[['filenames', 'popcorn', 'clump']])

                                    filenames popcorn  clump
0                         WS23_L1_T1_P_10.csv    None   None
1                         WS23_L3_T1_P_10.csv   False   True
2                          WS23_L3_T1_P_6.csv    True  False
3                          WS23_L3_T1_P_1.csv    True  False
4                          WS23_L3_T1_P_8.csv    True  False
5                         WS23_L1_T1_P_17.csv    None   None
6                         WS23_L3_T1_P_31.csv     NaN    NaN
7                          WS23_L1_T1_P_6.csv    None   None
8                          WS23_L1_T1_P_7.csv    None   None
9             WS25_Aug5_Loc1A_T1_F17_1029.csv     NaN    NaN
10             WS25_Aug5_Loc1A_T1_F6_0930.csv     NaN    NaN
11            WS25_Aug6_Loc1A_T1_F32_0714.csv     NaN    NaN
12            WS25_Aug5_Loc1A_T1_F12_1010.csv     NaN    NaN
13                         WS23_L3_T1_P_0.csv    None  False
14                         WS23_L1_T1_P_2.csv    None   None
15                      